In [1]:
import numpy as np
import open3d as o3d
import os
import cv2
import matplotlib.pyplot as plt
import json
import glob

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_dir = './2022-11-15'
output_data_dir = data_dir + '_instances'
os.makedirs(output_data_dir, exist_ok=True)
tiers = {
    # 'SongFeng':{

    # },
    'Toyota':{
        '21092302':{
            # '2022-04-18-rvbust_bright_lighting_lux1600',
            # '2022-04-18-rvbust_dark_lighting_lux5',
            # '2022-04-18-rvbust_normal_lighting_lux200',
            '2022-10-03-rvbust_synthetic'
        }
    },
    # 'ZSRobot':{
    #     '6010018CSV':{

    #     },
    #     '6010022CSV':{

    #     }
    # }

}

In [3]:
def extract_model_pcd(data_dir, company, part):
    model_path = os.path.join(data_dir, company, part, 'part_info', f'{part}.master.ply')
    print('\t\textract model point cloud from:', model_path)
    pcd = o3d.io.read_point_cloud(model_path)
    pcd = np.array(pcd.points)
    print('\t\tpoint cloud shape:', pcd.shape)
    ones = np.ones((pcd.shape[0], 1))
    pcd_one = np.concatenate([pcd, ones], axis=1)
    return pcd_one

In [4]:
def mask_instances(data_dir, company, part, condition, pcd_one):
    input_dir = os.path.join(data_dir, company, part, condition)
    output_dir = os.path.join(output_data_dir, company, part, condition)
    images = glob.glob(os.path.join(input_dir, 'depth','*.png'))
    images = [i.split('/')[-1].split('.')[0] for i in images]
    sorted(images)
    print(f'\t\t\tfound {len(images)} images in {input_dir}')

    for i in images:
        # depth_path = os.path.join(input_dir, 'depth', f'{i}.png')
        # color_path = os.path.join(input_dir, 'images', f'{i}.jpg')
        param_path = os.path.join(input_dir, 'images', f'{i}.json')
        label_path = os.path.join(input_dir, 'labels', f'{i}.npy')

        # read image
        # depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
        # color = cv2.imread(color_path, cv2.IMREAD_COLOR)
        
        # read json
        with open(param_path) as f:
            params = json.load(f)
        image_shape = np.array([params['height'], params['width']], dtype=int)
        K = np.array(params['K']).reshape((3,3))
        # read numpy
        labels = np.load(label_path)

        n_instances = len(labels)

        for j in range(n_instances):
            # read pose label from .npy
            pose = labels[j]
            # transform model points -> check depth
            points = np.matmul(pose, pcd_one.transpose())
            # project model points to image
            normalized_points = points[:3] / points[2]
            pixels = np.matmul(K, normalized_points).astype(int)[:2]
            pixels = np.unique(pixels, axis=1)
            pixels[0] = np.clip(pixels[0], 0, image_shape[1]-1)
            pixels[1] = np.clip(pixels[1], 0, image_shape[0]-1)
            pixels = np.unique(pixels, axis=1)
            # swap (x ,y) to (y, x)
            pixels[[0,1]] = pixels[[1,0]]

            y_min, y_max, x_min, x_max = pixels[0].min(), pixels[0].max(), pixels[1].min(), pixels[1].max()
            bbox = {
                'y_min': int(y_min),
                'y_max': int(y_max),
                'x_min': int(x_min),
                'x_max': int(x_max),
                'image_height': int(image_shape[0]),
                'image_width': int(image_shape[1])
            }

            flat_mask = np.zeros(image_shape).ravel()
            flat_index_array = np.ravel_multi_index(pixels, image_shape)
            flat_mask[flat_index_array] = 255
            mask = flat_mask.reshape(image_shape)


            file_name = i + f'_{j:04}'
            # save mask
            mask_dir = os.path.join(output_data_dir, company, part, condition, 'mask')
            os.makedirs(mask_dir, exist_ok=True)
            mask_path = os.path.join(mask_dir, file_name + '.png')
            cv2.imwrite(mask_path, mask)
            # save bbox
            bbox_dir = os.path.join(output_data_dir, company, part, condition, 'bbox')
            os.makedirs(bbox_dir, exist_ok=True)
            bbox_path = os.path.join(bbox_dir, file_name + '.json')
            with open(bbox_path, 'w') as f:
                json.dump(bbox,f)

    

In [5]:
if __name__ == '__main__':
    for company in tiers.keys():
        print('company:', company)
        for part in tiers[company].keys():
            print('\tpart:', part)
            pcd_one = extract_model_pcd(data_dir, company, part)
            for condition in tiers[company][part]:
                print('\t\tcondition:', condition)
                mask_instances(data_dir, company, part, condition, pcd_one)
                print('\t\tdone')

company: Toyota
	part: 21092302
		extract model point cloud from: ./2022-11-15/Toyota/21092302/part_info/21092302.master.ply
		point cloud shape: (54972, 3)
		condition: 2022-10-03-rvbust_synthetic
			found 4000 images in ./2022-11-15/Toyota/21092302/2022-10-03-rvbust_synthetic


KeyboardInterrupt: 